# Train and test models
## Big picture:
Train models to classify company descriptions into OpenSecrets industry labels. Test them on inputs in and out of domain.

## Basic setup

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import json
import nltk
import matplotlib.pyplot as plt
import random


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics


from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 


from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix

%matplotlib inline

In [ ]:
# DOES NOT NEED TO BE RUN EVERY TIME
# http://stackoverflow.com/questions/38128935/nltk-panlex-lite-giving-me-error/38135306#38135306
dler = nltk.downloader.Downloader()
dler._update_index()
dler._status_cache['panlex_lite'] = 'installed'
dler.download("all")

## Load data

In [2]:
companies = pd.read_csv('../data/lincoln/companies_catcodes_text.csv')

# just using values that have reuters for now
companies = companies[~companies.reuters.isnull()]

# create a binary classification for pharma
companies["isPharma"] = companies.apply(lambda row: 1 if row["Industry"]=="Pharmaceuticals/Health Products" else 0, axis=1)


## Some exploratory analysis

#### length before cutting out non-Reuters

In [3]:
len(pd.read_csv('../data/lincoln/companies_catcodes_text.csv'))

3623

#### length after cutting out non-Reuters


In [4]:
len(companies)

3416

In [5]:
len(companies[companies.Industry == "Pharmaceuticals/Health Products"])

545

In [6]:
companies["Industry"].value_counts()

Pharmaceuticals/Health Products      545
Electronics Mfg & Equip              395
Commercial Banks                     345
Oil & Gas                            253
Real Estate                          241
Misc Manufacturing & Distributing    201
Business Services                    163
Insurance                            139
Savings & Loans                       91
Retail Sales                          86
Telecom Services                      80
Electric Utilities                    75
Securities & Investment               73
Telephone Utilities                   69
Food & Beverage                       58
TV/Movies/Music                       54
Automotive                            46
Air Transport                         41
Sea Transport                         39
Chemical & Related Manufacturing      39
Lodging/Tourism                       26
Finance/Credit Companies              26
Health Services/HMOs                  26
Food Processing & Sales               26
Trucking        

## Useful functions for splitting out data to work with

If possible, return a dataframe of size size on which row_fn is true, and a dataframe of size size on which row_fn is false

In [7]:
def get_equal_samples(df, row_fn, size):
    
    b = df.apply(row_fn, axis = 1)
    
    yes = df[b]
    no = df[~b]

    if len(yes) < size or len(no) < size:
        return None, None
    
    yes_sample = yes.sample(size, replace = False)
    no_sample = no.sample(size, replace = False)
    
    return yes_sample, no_sample

Concatenate and shuffle two dataframes, returning one

In [8]:
def mix_and_shuffle(d1, d2):
    d = pd.concat([d1,d2]).sample(frac=1)
    return d

## Make an isPharma dataset

In [9]:
ypharma, npharma = get_equal_samples(companies, lambda row: row['isPharma'] == 1, 500)
allpharma = mix_and_shuffle(ypharma, npharma)

## Some basic feature construction and modeling work in NLTK
#### Mostly adapted from http://www.nltk.org/book/ch06.html
With a lot of help from:

http://stackoverflow.com/questions/20827741/nltk-naivebayesclassifier-training-for-sentiment-analysis

http://www.nltk.org/book/ch01.html

http://www.nltk.org/book/ch03.html

In [10]:
text, label = list(allpharma["reuters"]), list(allpharma["isPharma"])
documents = list(zip(text, label))

In [11]:
all_text = " ".join(text)
t = nltk.word_tokenize(all_text)
fdmc = nltk.FreqDist(t).most_common(2000)

In [12]:
def document_features(document, word_features):
    document_words = set(nltk.word_tokenize(document))
    features = {}
    for word in word_features:
        features['contains({})'.format(word[0])] = (word[0] in document_words)
    return features

In [13]:
featuresets = [(document_features(d, fdmc), c) for (d,c) in documents]
train_set, test_set = featuresets[500:], featuresets[:500]

In [14]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [15]:
print(nltk.classify.accuracy(classifier, test_set))

0.974


97% classification isn't bad for such a basic attempt, though it's a very simple task (isPharma), and distribution of classes probably not very even.

In [16]:
classifier.show_most_informative_features(20)

Most Informative Features
    contains(candidates) = True                1 : 0      =     61.7 : 1.0
contains(biopharmaceutical) = True                1 : 0      =     57.8 : 1.0
contains(Pharmaceuticals) = True                1 : 0      =     56.5 : 1.0
    contains(investment) = True                0 : 1      =     54.0 : 1.0
          contains(real) = True                0 : 1      =     53.3 : 1.0
   contains(residential) = True                0 : 1      =     52.7 : 1.0
  contains(Therapeutics) = True                1 : 0      =     51.3 : 1.0
       contains(million) = True                0 : 1      =     43.0 : 1.0
     contains(candidate) = True                1 : 0      =     39.3 : 1.0
        contains(credit) = True                0 : 1      =     38.2 : 1.0
         contains(Phase) = True                1 : 0      =     35.3 : 1.0
       contains(therapy) = True                1 : 0      =     33.9 : 1.0
          contains(Bank) = True                0 : 1      =     32.7 :

The above features are fairly unsurprising. The words "candidates", "biopharmaceutical", and "Pharmaceuticals" are the words most positively correlated with the pharmaceutical industry, while "investment", "real", and "residential" are the words most positively associated with belonging to an industry other than pharmaceutical.

### Just playing around with this classifier a bit

In [17]:
classifier.prob_classify(document_features("finance construction financial deposits residential ",fdmc)).prob(1)

0.3957733053102362

...sure. Kinda high, but sure.

In [18]:
classifier.prob_classify(document_features("health drug",fdmc)).prob(1)

0.9999999987357053

...sure.

In [19]:
classifier.prob_classify(document_features("nonsense blah blah",fdmc)).prob(1)

0.9999999765936857

...hmm. This is a less pleasing result.

In [20]:
out = classifier.classify_many([t[0] for t in test_set])
print(ConfusionMatrix ([t[1] for t in test_set], out))

  |   0   1 |
--+---------+
0 |<254>  . |
1 |  13<233>|
--+---------+
(row = reference; col = test)



Okay, so isPharma classifier clearly works reasonably well on data within the domain, but does not work well on data from a different distribution (like my "nonsense blah blah" example, which is given a 0.99 toward pharma-related).

## Switching to sklearn
### Turns out we can do pretty much all the above (and more easily) in sklearn
#### Adapted from:
http://bbengfort.github.io/tutorials/2016/05/19/text-classification-nltk-sckit-learn.html

http://nlpforhackers.io/building-a-nlp-pipeline-in-nltk/

http://nlpforhackers.io/text-classification/

http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html

http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

http://scikit-learn.org/stable/modules/feature_extraction.html

http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html


#### Lemmatizer from http://scikit-learn.org/stable/modules/feature_extraction.html 
Tested, but didn't seem to add any lift

In [21]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

#### Helper function 
that given a pipeline and training and testing data, trains and returns the model (printing test set error). Inspired by: http://nlpforhackers.io/text-classification/

In [22]:
def train_and_predict(model, data, model_name = None):
    m = model.fit(data['X_train'], data['y_train'])
    predicted = m.predict(data['X_test'])
    c_rate = np.mean(predicted == data['y_test'])
    if model_name is None:
        print(c_rate)
    else:
        print(model_name + ": " + str(c_rate))
    return m

### Define our model pipelines
(In a function so can be reset easily)

In [23]:
def reset_models():
    
    # MULTINOMIAL NAIVE BAYES
    MNB = Pipeline([
            ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
    #         ('tfidf', TfidfTransformer()),
            ('clf', MultinomialNB())
    ])
    
    
    # BAGGING CLASSIFIER WITH DECISION TREES
    BDT = Pipeline([
            ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
            ('tfidf', TfidfTransformer()),
            ('clf', BaggingClassifier())
    ])
    
    
    # SUPPORT VECTOR MACHINE
    SVM = Pipeline([
            ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge'))
    ])
    
    # LOGISTIC REGRESSION
    LOG = Pipeline([
            ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='log'))
    ])
    
    return {'MNB':MNB, 'BDT':BDT, 'SVM':SVM, 'LOG':LOG}

### Split out testing and training data for three tasks

In [24]:
isPharma_data = {}
isPharma_data['X_train'], isPharma_data['X_test'], isPharma_data['y_train'], isPharma_data['y_test'] = train_test_split(companies.reuters.values, 
                                                    companies['isPharma'].values,
                                                   test_size=0.2)

In [25]:
sector_data = {}
sector_data['X_train'], sector_data['X_test'], sector_data['y_train'], sector_data['y_test'] = train_test_split(companies.reuters.values, 
                                                    companies["Sector Long"].values,
                                                   test_size=0.2)

In [26]:
industry_data = {}
industry_data['X_train'], industry_data['X_test'], industry_data['y_train'], industry_data['y_test'] = train_test_split(companies.reuters.values, 
                                                    companies["Industry"].values,
                                                   test_size=0.2)

### Train models for each task

In [27]:
isPharma_models = {k: train_and_predict(v, isPharma_data, k) for k,v in reset_models().items()}

BDT: 0.960526315789
LOG: 0.979532163743
MNB: 0.985380116959
SVM: 0.983918128655


In [28]:
sector_models = {k: train_and_predict(v, sector_data, k) for k,v in reset_models().items()}

BDT: 0.812865497076
LOG: 0.891812865497
MNB: 0.861111111111
SVM: 0.893274853801


In [29]:
industry_models = {k: train_and_predict(v, industry_data, k) for k,v in reset_models().items()}

BDT: 0.703216374269
LOG: 0.795321637427
MNB: 0.707602339181
SVM: 0.81432748538


### Apply results

Helper function that prints potential results above a threshold for a list of texts (potentially only from certain classes -- useful for situation where looking for conflicts of interest)

In [30]:
def cats_above_threshhold(model, text, threshold = -1, notable_classes = None):
    if isinstance(text, str):
        text = [text]
    results = model.predict_proba(text)
    for i,r in enumerate(results):
        zipped = list(zip(model.classes_, r))
        for z in zipped:
            if z[1] >= threshold:
                if notable_classes is None or z[0] in notable_classes:
                    print("TEXT: " + text[i] + " | LABEL: " + str(z[0]) + " | PROBABILITY: " + str(z[1]))

Example:

In [31]:
cats_above_threshhold(industry_models['LOG'], ["pharmaceutical", 'bank finance','nonsense blah'], .1)

TEXT: pharmaceutical | LABEL: Pharmaceuticals/Health Products | PROBABILITY: 0.334560977674
TEXT: bank finance | LABEL: Commercial Banks | PROBABILITY: 0.311069724997


### old

In [ ]:
# SECTOR
mnb = train_and_predict(mnb, X_train, y_train, X_test, y_test)
bc = train_and_predict(bc, X_train, y_train, X_test, y_test)
sgd_log = train_and_predict(sgd_log, X_train, y_train, X_test, y_test)
sgd_hinge = train_and_predict(sgd_hinge, X_train, y_train, X_test, y_test)

In [ ]:
# INDUSTRY
mnb = train_and_predict(mnb, X_train, y_train, X_test, y_test)
bc = train_and_predict(bc, X_train, y_train, X_test, y_test)
sgd_log = train_and_predict(sgd_log, X_train, y_train, X_test, y_test)
sgd_hinge = train_and_predict(sgd_hinge, X_train, y_train, X_test, y_test)

## Now we'll try models on some tweets

### Read in tweets, and twitter_ids

In [32]:
data_relative_path = "../data/lincoln/"
streamed_tweets_location = data_relative_path + "congress_streamed_tweets/"
searched_tweets_location = data_relative_path + "congress_searched_tweets/"

In [33]:
# GET THE USER IDS
df = pd.read_csv(data_relative_path + "current_social_media.csv", dtype=str)
user_ids = df.twitter_id
good_user_ids = []
for uid in user_ids:
    try:
        # THIS IS JUST SO WE IGNORE NANS VALUES
        g = int(uid)
        good_user_ids.append(uid)
    except ValueError:
        pass

### Just as an example...

In [34]:
# just for testing
temp_ids = [good_user_ids[1]]

In [35]:
temp_ids

['117501995']

In [36]:
for guid in temp_ids:
    # http://stackoverflow.com/questions/21058935/python-json-loads-shows-valueerror-extra-data
    tweets = []
    for line in open(searched_tweets_location + guid + '.json','r'):
        tweets.append(json.loads(line))
    print(len(tweets))

3199


In [37]:
tweet_text = [t['text'] for t in tweets]

### Proof of concept

In [38]:
cats_above_threshhold(industry_models['BDT'], tweet_text, .8)

TEXT: We must manage our #PublicLands for the benefit of all Americans — not just #oil #gas &amp; #mining companies’ interests https://t.co/d2Mn4gwZnI | LABEL: Oil & Gas | PROBABILITY: 0.9
TEXT: We must manage our #PublicLands for the benefit of all Americans — not just #oil #gas &amp; #mining companies’ interests https://t.co/d2Mn4gwZnI | LABEL: Oil & Gas | PROBABILITY: 0.9
TEXT: Gorsuch has ruled against #LGBTQ individuals seeking fair, nondiscriminatory treatment. I will not support such hostility towards our rights | LABEL: Pharmaceuticals/Health Products | PROBABILITY: 0.9
TEXT: Trump’s #EO fails in creating rural #cleanenergy jobs, ignores impacts extreme weather will have &amp; doesn't decrease our foreign oil reliance | LABEL: Oil & Gas | PROBABILITY: 0.8
TEXT: Republican #ACA repeal &amp; disastrous #AHCA will leave millions w/o coverage including for addiction treatment in mid… https://t.co/2mhK9EGYIU | LABEL: Pharmaceuticals/Health Products | PROBABILITY: 1.0
TEXT: At the he

### Investigating a tweet a little further

Given a score of 1.0 for 'Sea Transport' by BDT

In [39]:
sample2 = "Maritime listening session: Tom Albro, @PortofSeattle President says ½ of total U.S. catch is from vessels that call Seattle home."

All but MNB call it 'Sea Transport'

In [40]:
for k,v in industry_models.items():
    print (k + ":\t" + str(v.predict([sample2])[0]))

BDT:	Sea Transport
LOG:	Sea Transport
MNB:	Telecom Services
SVM:	Sea Transport


But only gets a 0.0909 for 'Sea Transport' for SVM, despite that being the top choice.

In [41]:
list(zip(industry_models['LOG'].classes_, industry_models['LOG'].predict_proba([sample2])[0]))

[('Agricultural Services/Products', 0.016306075422533258),
 ('Air Transport', 0.017767566288701523),
 ('Automotive', 0.016785773477096842),
 ('Beer, Wine & Liquor', 0.014543140339573083),
 ('Building Materials & Equipment', 0.019180511374422466),
 ('Business Services', 0.021792513621531916),
 ('Casinos/Gambling', 0.014651109528171386),
 ('Chemical & Related Manufacturing', 0.017902814411376974),
 ('Commercial Banks', 0.015961804292846386),
 ('Construction Services', 0.013332173721641572),
 ('Electric Utilities', 0.018485654119863467),
 ('Electronics Mfg & Equip', 0.040993951716752144),
 ('Finance/Credit Companies', 0.014131133374126482),
 ('Food & Beverage', 0.021131613134399069),
 ('Food Processing & Sales', 0.017186906834030066),
 ('Forestry & Forest Products', 0.016030085912031315),
 ('General Contractors', 0.013734313982763152),
 ('Health Professionals', 0.013934770015045729),
 ('Health Services/HMOs', 0.017928321996977477),
 ('Home Builders', 0.017148387999421963),
 ('Hospitals/Nu

Not that many samples! (Though still top half of categories, unfortunately.)

In [42]:
len(companies[companies.Industry=="Sea Transport"])

39

## Add in donations data

In [43]:
with open("../data/donations/donations_industry.json", 'r') as f:
    donations_industry = json.load(f)

For all given twitter_ids, highlight high-scoring ones (only within high-donation industries if requested)

In [44]:
def locate_conflicts(twitter_ids = [], filter_by_donations = True, model = None, threshold = -1):
    if isinstance(twitter_ids, str):
        twitter_ids = [twitter_ids]
        
    for t in twitter_ids:
        
        try:
            print("---USER: " + t + "---")

            # gather relevant industry names (sources of potential conflicts of interest)
            ilist = donations_industry[df[df.twitter_id == t]["opensecrets"].values[0]]['response']['industries']['industry'] 
            inames = [i['@attributes']['industry_name'] for i in ilist]

            # gather all tweets
            tweets = []
            for line in open(searched_tweets_location + t + '.json','r'):
                tweets.append(json.loads(line))
            tweet_text = [t['text'] for t in tweets]

            print("TOTAL # OF TWEETS: " + str(len(tweet_text)))
            print("")

            if len(tweet_text) == 0:
                continue
        
            # score tweets and print relevant ones        
            if filter_by_donations:
                cats_above_threshhold(model, tweet_text, threshold, inames)
            else:
                cats_above_threshhold(model, tweet_text, threshold)
        except:
            print("ERROR ON TWITTER_ID: " + t)
        
        print("")

### Some sample experimentation

Some tweets which may not necessarily be conflicts of interest...

In [45]:
locate_conflicts(good_user_ids[:20], True, industry_models['BDT'], 1.0)

---USER: 43910797---
TOTAL # OF TWEETS: 2865

TEXT: Shameful for politicians with taxpayer-funded healthcare to ‘gamble’ with the insurance of 900,000 Ohioans. https://t.co/h3UHmxNAaB | LABEL: Insurance | PROBABILITY: 1.0
TEXT: Dan was billed nearly $17k for his ER care—despite having health insurance. My new legislation will protect folks f… https://t.co/AxB69Ed6Y1 | LABEL: Insurance | PROBABILITY: 1.0
TEXT: RT @WhiteHouse: "So do another 3 million children" —@POTUS on kids who have gained insurance since he took office https://t.co/3uyu39axca | LABEL: Insurance | PROBABILITY: 1.0
TEXT: TY #SCOTUS for reaffirming what health law is about: helping families purchase affordable health insurance. #ACAworks http://t.co/m3dRSGpzRx | LABEL: Insurance | PROBABILITY: 1.0
TEXT: New @BPC_Bipartisan report calls for extension of Children's Health Insurance. Our bill would extend for 4 years: http://t.co/QVsM46As4T | LABEL: Insurance | PROBABILITY: 1.0
TEXT: All Americans deserve access to primary

In [46]:
locate_conflicts(good_user_ids[:5], True, industry_models['LOG'], .3)

---USER: 43910797---
TOTAL # OF TWEETS: 2865


---USER: 117501995---
TOTAL # OF TWEETS: 3199


---USER: 109071031---
TOTAL # OF TWEETS: 3220


---USER: 249787913---
TOTAL # OF TWEETS: 3238

TEXT: These actions by @POTUS will expand access to treatment for prescription drug and #opioid addiction https://t.co/HpxAS1aZVA | LABEL: Pharmaceuticals/Health Products | PROBABILITY: 0.305329491434
TEXT: To address this epidemic, we need to focus on prevention, treatment &amp; access to treatment for those suffering with substance abuse disorders | LABEL: Pharmaceuticals/Health Products | PROBABILITY: 0.375773427939

---USER: 171598736---
TOTAL # OF TWEETS: 3246




## Debugging a few specific instances

In [47]:
tweets = []
for line in open(searched_tweets_location + "829794295355940866" + '.json','r'):
    tweets.append(json.loads(line))
len(tweets)
ilist = donations_industry[df[df.twitter_id == "829794295355940866"]["opensecrets"].values[0]]['response']['industries']['industry'] 
inames = [i['@attributes']['industry_name'] for i in ilist]

KeyError: nan

In [48]:
df[df.twitter_id == "829794295355940866"]

,bioguide,name,opensecrets,rss_url,state,twitter,twitter_id
535,S001202,Luther Strange,NaN,NaN,AL,SenatorStrange,829794295355940866


Just Luther Strange causing trouble again...